# MIDS - w261 Machine Learning At Scale
__Course Lead:__ Dr James G. Shanahan (__email__ Jimi via  James.Shanahan _AT_ gmail.com)

## Assignment - HW5


---
__Name:__  Chris Caudill   
__Class:__ MIDS w261 (Section Spring 2017 Group 1)     
__Email:__  cscaudill@iSchool.Berkeley.edu     
__StudentId__  3032134574    __End of StudentId__     
__Week:__   5

__NOTE:__ please replace `1234567` with your student id above      
__Due Time:__ HW is due the Tuesday of the following week by 8AM (West coast time). I.e., Tuesday, Feb 14, 2017 in the case of this homework. 

* __HW5 Phase 1__ 
This can be done on a local machine (with a unit test on the cloud such as AltaScale's PaaS or on AWS) and is due Tuesday, Week 6 by 8AM (West coast time). It will primarily focus on building a unit/systems and for pairwise similarity calculations pipeline (for stripe documents)

* __HW5 Phase 2__ 
This will require the AltaScale cluster and will be due Tuesday, Feb 21 by 8AM (West coast time). 
The focus of  HW5 Phase 2  will be to scale up the unit/systems tests to the Google 5 gram corpus. This will be a group exercise 


# Table of Contents <a name="TOC"></a> 

1.  [HW Intructions](#1)   
2.  [HW References](#2)
3.  [HW Problems](#3)   
1.  [HW Introduction](#1)   
2.  [HW References](#2)
3.  [HW  Problems](#3)   
    5.0.  [HW5.0](#5.0)   
    5.1.  [HW5.1](#5.1)   
    5.2.  [HW5.2](#5.2)   
    5.3.  [HW5.3](#5.3)    
    
   

<a name="1"></a>
# 1 Instructions
[Back to Table of Contents](#TOC)

MIDS UC Berkeley, Machine Learning at Scale   
DATSCIW261 ASSIGNMENT #5

Version 2017-9-2 


### IMPORTANT

This homework can be completed locally on your computer 

### === INSTRUCTIONS for SUBMISSIONS ===   
Follow the instructions for submissions carefully.

Each student has a `HW-<user>` repository for all assignments.   

Click this link to enable you to create a github repo within the MIDS261 Classroom:   
https://classroom.github.com/assignment-invitations/3b1d6c8e58351209f9dd865537111ff8   
and follow the instructions to create a HW repo.

Push the following to your HW github repo into the master branch:
* Your local HW5 directory. Your repo file structure should look like this:

```
HW-<user>
    --HW3
       |__MIDS-W261-HW-03-<Student_id>.ipynb
       |__MIDS-W261-HW-03-<Student_id>.pdf
       |__some other hw3 file
    --HW4
       |__MIDS-W261-HW-04-<Student_id>.ipynb
       |__MIDS-W261-HW-04-<Student_id>.pdf
       |__some other hw4 file
    etc..
```    

  
<a name="2">
# 2 Useful References
[Back to Table of Contents](#TOC)

* See async and live lectures for this week

<a name="3">
# HW Problems
[Back to Table of Contents](#TOC)

## 3.  HW5.0  data warehouse; star schema<a name="5.0"></a>
[Back to Table of Contents](#TOC)

- What is a data warehouse? What is a Star schema? When is it used?

A Data Warehouse is a repository of data from a variety of sources and is typically used in an enterprise setting as the basis for Business Intelligence and other types of data analysis. DWs can contain a range of data from structured to unstructured.

Star Schema is an approach used in Data Warehouses where a fact table is used as a compound reference of multiple dimension tables. This is used to create a single table that can represent the relation between two dimension tables. For example, if we own a business with multiple locations, our data repository may contain a table for product inventory, customers, and store location. Each of these 3 tables would be dimensions. The fact table would contain foreign keys that reference the associated product/customer/store from their respective table.

## 3.  HW5.1 Databases: 3NF; denormalized <a name="5.1"></a>
[Back to Table of Contents](#TOC)

- In the database world What is 3NF? Does machine learning use data in 3NF? If so why? 
- In what form does ML consume data?
- Why would one use log files that are denormalized?

- 3NF (Third Normal Form) is format that data is typically stored within a database where data from multiple tables is represented in smaller tables by their keys. 3NF does not usually contain all of the fields from the referenced tables and therefore in Machine Learning, we will often need to denormalize the data to get the complete picture. If the 3NF table contains all of the necessary information, it may be used, but this is not always the case.
- ML can technically consume data in 1st, 2nd, or 3rd normal form. As mentioned above, if the normalized data contains the required content, denormalization may not be necessary. In an ideal scenario, ML would have all of the required content within each record (3NF), but due to technologies such as Hadoop, we can perform massive joins much more efficiently than we could with traditional databases. 
- Denormalized log files are benefical because a typical log file can have a mass amount of rows. If we were required to reference other tables for additional data associated with each line item, we would be much less efficient than if we were to simply have all of the required content within each line.

## 3.  HW5.2  Memory-backed map-side<a name="5.2"></a>
[Back to Table of Contents](#TOC)

Using MRJob, implement a hashside join (memory-backed map-side) for left, right and inner joins. Use the following tables for this HW and join based on the country code (third column of the transactions table and the second column of the Countries table:

<PRE>
transactions.dat
Alice Bob|$10|US
Sam Sneed|$1|CA
Jon Sneed|$20|CA
Arnold Wesise|$400|UK
Henry Bob|$2|US
Yo Yo Ma|$2|CA
Jon York|$44|CA
Alex Ball|$5|UK
Jim Davis|$66|JA

Countries.dat
United States|US
Canada|CA
United Kingdom|UK
Italy|IT

</PRE>

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

- (1) Left joining Table Left with Table Right
- (2) Right joining Table Left with Table Right
- (3) Inner joining Table Left with Table Right


In [251]:
%%writefile transactions.txt
Alice Bob|$10|US
Sam Sneed|$1|CA
Jon Sneed|$20|CA
Arnold Wesise|$400|UK
Henry Bob|$2|US
Yo Yo Ma|$2|CA
Jon York|$44|CA
Alex Ball|$5|UK
Jim Davis|$66|JA

Overwriting transactions.txt


In [252]:
%%writefile countries.txt
United States|US
Canada|CA
United Kingdom|UK
Italy|IT

Overwriting countries.txt


**For the LEFT join, we output all of the rows from the TRANSACTIONS file, regardless of whether they have a match in the COUNTRIES file. If there is a match, we join the rows.**

**The Reason that I chose to use the TRANSACTIONS file as my LEFT table is because it logically made more sense. The table on the left will output ALL of its rows regardless of a match in the RIGHT table or not. In the scenario where we have a TRANSACTIONS file and a COUNTRIES file, it seemed to me that a typical use case would be more likely to be interested in joining the long description of the country onto each transaction, as opposed to the other way around.**

**Also, I chose to load my COUNTRIES file into memory as it was the smaller of the two tables.**

In [14]:
%%writefile leftHashJoin.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import jobconf_from_env
import os

class leftJoin(MRJob):
    
    def steps(self):
        countries=[]
        return [
            MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final
            )]
    
    def mapper_init(self):
        countries = open(str(os.path.dirname(os.path.realpath(__file__)))+"/countries.txt","r")
        self.left=0
        self.nomatch=0
        self.ct = {}
        for line in countries:
            ct_long, ct_short  = line.split('|',1)
            self.ct[ct_short.strip()] = ct_long.strip()         
 
    def mapper(self, _, line):
        cust,price,country = line.split('|',2)
        
        # In this case, we are left joining with the transactions file on the left. 
        # Therefore, we will output 9 rows regardless of how many matches show up in the countries file.
        
        if country in self.ct:
            self.left += 1
            yield None, country+","+self.ct[country]+","+cust+","+price
        else:
            self.nomatch += 1
            yield None, country+",NULL,"+cust+","+price
            
            
    # Mapper_Final will output the total joined rows
    def mapper_final(self):
        yield None, "left-joined "+str(self.left)+" row(s)."
        yield None, "No left match for "+str(self.nomatch)+" row(s)."
        yield None, "Total Rows: "+str(self.left + self.nomatch)
        

if __name__ == '__main__':
    leftJoin.run()

Overwriting leftHashJoin.py


**For the RIGHT join, we output all of the rows from the COUNTRIES file, whether or not there is a match in the TRANSACTIONS file. If there is a match, we join them.**

In [15]:
%%writefile rightHashJoin.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import jobconf_from_env
import os

class rightJoin(MRJob):
    
    def steps(self):
        return [
            MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final
            )]
    
    def mapper_init(self):
        countries = open(str(os.path.dirname(os.path.realpath(__file__)))+"/countries.txt","r")
        self.right=0
        self.nomatch=0
        self.ct = {}
        self.matched_ct={}
        for line in countries:
            ct_long, ct_short  = line.split('|',1)
            self.ct[ct_short.strip()] = ct_long.strip()         
#             self.matched_ct[ct_short.strip()] = ct_long.strip()
            
    def mapper(self, _, line):
        cust,price,country = line.split('|',2)
        
        # In this case, we are right joining with the transactions file on the left. 
        # Therefore, we will output all of the rows from the countries table, and then any rows
        # from the transactions table that have a country match
        
        if country in self.ct:
            self.right += 1
            yield None, country+","+self.ct[country]+","+cust+","+price
            self.matched_ct[country] = self.ct[country]

            
            
    # Mapper_Final will output the total joined rows
    def mapper_final(self):
        for key,value in self.ct.iteritems():
            if key not in self.matched_ct:
                self.nomatch += 1
                yield None, key+","+value+",NULL,NULL"
        yield None, "right-joined "+str(self.right)+" row(s)."
        yield None, "No right match for "+str(self.nomatch)+" row(s)."
        yield None, "Total Rows: "+str(self.right + self.nomatch)        

if __name__ == '__main__':
    rightJoin.run()

Overwriting rightHashJoin.py


**For the INNER join, we output all of the records that match between the two files.**

In [16]:
%%writefile innerHashJoin.py
#!/usr/bin/env python

from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.compat import jobconf_from_env
import os

class innerJoin(MRJob):
    
    def steps(self):
        countries=[]
        return [
            MRStep(
                mapper_init = self.mapper_init,
                mapper = self.mapper,
                mapper_final = self.mapper_final
            )]
    
    def mapper_init(self):
        countries = open(str(os.path.dirname(os.path.realpath(__file__)))+"/countries.txt","r")
        self.inner=0
        self.ct = {}
        for line in countries:
            ct_long, ct_short  = line.split('|',1)
            self.ct[ct_short.strip()] = ct_long.strip()         
 
    def mapper(self, _, line):
        cust,price,country = line.split('|',2)
        if country in self.ct:
            self.inner += 1
            yield None, self.ct[country]+","+cust+","+price
            
    def mapper_final(self):
        yield None, "inner-joined "+str(self.inner)+" rows."
        
        
if __name__ == '__main__':
    innerJoin.run()

Overwriting innerHashJoin.py


In [17]:
!chmod a+x leftHashJoin.py
!./leftHashJoin.py --jobconf mapred.map.tasks=1 transactions.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/leftHashJoin.root.20170214.034404.240845
Running step 1 of 1...
Streaming final output from /tmp/leftHashJoin.root.20170214.034404.240845/output...
null	"US,United States,Alice Bob,$10"
null	"CA,Canada,Sam Sneed,$1"
null	"CA,Canada,Jon Sneed,$20"
null	"UK,United Kingdom,Arnold Wesise,$400"
null	"US,United States,Henry Bob,$2"
null	"CA,Canada,Yo Yo Ma,$2"
null	"CA,Canada,Jon York,$44"
null	"UK,United Kingdom,Alex Ball,$5"
null	"JA,NULL,Jim Davis,$66"
null	"left-joined 8 row(s)."
null	"No left match for 1 row(s)."
null	"Total Rows: 9"
Removing temp directory /tmp/leftHashJoin.root.20170214.034404.240845...


In [18]:
!chmod a+x rightHashJoin.py
!./rightHashJoin.py --jobconf mapred.map.tasks=1 transactions.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/rightHashJoin.root.20170214.034405.650385
Running step 1 of 1...
Streaming final output from /tmp/rightHashJoin.root.20170214.034405.650385/output...
null	"US,United States,Alice Bob,$10"
null	"CA,Canada,Sam Sneed,$1"
null	"CA,Canada,Jon Sneed,$20"
null	"UK,United Kingdom,Arnold Wesise,$400"
null	"US,United States,Henry Bob,$2"
null	"CA,Canada,Yo Yo Ma,$2"
null	"CA,Canada,Jon York,$44"
null	"UK,United Kingdom,Alex Ball,$5"
null	"IT,Italy,NULL,NULL"
null	"right-joined 8 row(s)."
null	"No right match for 1 row(s)."
null	"Total Rows: 9"
Removing temp directory /tmp/rightHashJoin.root.20170214.034405.650385...


In [19]:
!chmod a+x innerHashJoin.py
!./innerHashJoin.py --jobconf mapred.map.tasks=1 transactions.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/innerHashJoin.root.20170214.034407.159333
Running step 1 of 1...
Streaming final output from /tmp/innerHashJoin.root.20170214.034407.159333/output...
null	"United States,Alice Bob,$10"
null	"Canada,Sam Sneed,$1"
null	"Canada,Jon Sneed,$20"
null	"United Kingdom,Arnold Wesise,$400"
null	"United States,Henry Bob,$2"
null	"Canada,Yo Yo Ma,$2"
null	"Canada,Jon York,$44"
null	"United Kingdom,Alex Ball,$5"
null	"inner-joined 8 rows."
Removing temp directory /tmp/innerHashJoin.root.20170214.034407.159333...


## 3.  HW5.2.1 (OPTIONAL) Almost stateless reducer-side join  <a name="5.2.1"></a>
[Back to Table of Contents](#TOC)

The following MRJob code, implements a reduce-side join for an inner join. The reducer is almost stateless, i.e., uses as little memory as possible. Use the tables from HW5.2 for this HW and join based on the country code (third column of the transactions table and the second column of the Countries table perform. Perform  an left, right, inner joins using the code provided below and report the number of rows resulting from:

- (1) Left joining Table Left with Table Right
- (2) Right joining Table Left with Table Right
- (3) Inner joining Table Left with Table Right

Again make smart decisions about which table should be the left table (i.e., crosscheck the code). 

__Some notes on the code__ 
Here, the mapper receives its set of input splits either from the transaction table or from the countries table and makes the appropriate transformations: splitting the line into fields, and emitting a key/value. The key is the join key - in this case, the country code field of both sets of records. The mapper knows which file and type of record it is receiving based on the length of the fields. The records it emits contain the join field as the key, which acts as the partitioning key; We use the SORT_VALUES option, which ensures the values are sorted as well. Then, we employ a trick to ensure that for each join key, country records are seen always before transaction records. We achieve this by adding an arbitrary key to the front of the value: 'A' for countries, 'B' for customers. This makes countries sort before customers for each and every join/partition key. After that trick, the join is simply a matter of storing countries ('A' records) and crossing this array with each customer record.

In [ ]:
import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):

  # Performs secondary sort
  SORT_VALUES = True

  def mapper(self, _, line):
    splits = line.rstrip("\n").split("|")

    if len(splits) == 2: # country data
      symbol = 'A' # make country sort before transaction data
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    else: # person data
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]

  def reducer(self, key, values):
    countries = [] # should come first, as they are sorted on artificia key 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
          yield key, country[1:] + value[1:]

if __name__ == '__main__':
  MRJoin.run()

# 5.3 Pairwise similarity  - PHASE 1 <a name="5.3"></a>

In this part of the assignment we will focus on developing methods for detecting synonyms, using the Google 5-grams dataset. To accomplish this you must script two main tasks using MRJob:


#### (1) Using the systems tests data sets, write mrjob code to build the stripes
#### (2) Write mrjob code to build an inverted index from the stripes
#### (3) Using two (symmetric) comparison methods of your choice (e.g., correlations, distances, similarities), pairwise compare all stripes (vectors), and output to a file.   

__==Design notes for (1)== __  
For this task you will be able to modify the pattern we used in HW 3.2 (feel free to use the solution as reference). To total the word counts across the n-grams, output the support from the mappers using the total order inversion pattern:

<*word,count>   

to ensure that the support arrives before the cooccurrences.   

In addition to ensuring the determination of the total word counts, the mapper must also output co-occurrence counts for the pairs of words inside of each n-gram. Treat these words as a basket, as we have in HW 3, but count all stripes or pairs in both orders, i.e., count both orderings: (word1,word2), and (word2,word1), to preserve
symmetry in our output for (2).

__==Design notes for (3)==__   
For this task you will have to determine a method of comparison.
Here are a few that you might consider:

 - Jaccard
 - Cosine similarity
 - Spearman correlation
 - Euclidean distance
 - Taxicab (Manhattan) distance
 - Shortest path graph distance (a graph, because our data is symmetric!)
 - Pearson correlation
 - Kendall correlation
 ...

However, be cautioned that some comparison methods are more difficult to parallelize than others, and do not perform more associations than is necessary, since your choice of association will be symmetric.

Please use the inverted index (discussed in live session #5) based pattern to compute the pairwise (term-by-term) similarity matrix. 

In [291]:
%%writefile buildStripes.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import re
import mrjob
import json
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRbuildStripes(MRJob):
            
    def mapper(self, _, line):
        ngram,count,page_count,book_count = line.split('\t',3)
        ngram = ngram.strip()
        count = int(count)
        
        # lowercase and parse out each word
        words = ngram.lower().split()
        
        d = {}
        
        # Create a dictionary within a dictionary
        # For example: d[biography] = {"a": 92, "of": 92, "george": 92, "general": 92}
        
        for term in sorted(words):
            if term not in d.keys():
                d[term] = {}

            for term2 in sorted(words):
                if term != term2:
                    if term2 in d[term]:
                        d[term][term2] += count
                    else:
                        d[term][term2] = count
                        
        # iterate through the dictionary and yield the top level term, the second term, and the cooccurrence count
        # Example: "biography, (general, 92)"
        
        for k,v in d.iteritems():
            for k2,v2 in d[k].iteritems():
                yield k, (k2, v2)
            
    
    def reducer(self, key, line):
        
        red_d = {}
        term1 = key
        
        # Combine the various term cooccurrence counts into a single dictionary
        
        for term,count in line:
            count = int(count)
            term2 = term
            if term1 not in red_d.keys():
                red_d[term1] = {}
            if term2 in red_d[term1]:
                red_d[term1][term2] += count
            else:
                red_d[term1][term2] = count

        for k,v in red_d.iteritems():
            yield k,v
  
  #END SUDENT CODE531_STRIPES
if __name__ == '__main__':
    MRbuildStripes.run()

Overwriting buildStripes.py


In [292]:
%%writefile invertedIndex.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-


from __future__ import division
import collections
import re
import json
import math
# import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep
import ast

class MRinvertedIndex(MRJob):
    
#START STUDENT CODE531_INV_INDEX

    def mapper(self,_,line):
        
        line = line.strip()
        key_term, words = line.split("\t")
        
        # 'words' are coming in with the structure of a dictionary, but formatted as a string
        # ast.literal_eval converts it to the dictionary that it should be
        words = ast.literal_eval(words)
        _len = len(words)
        
        # for each word, output the cooccurring terms and the number of associated cooccurring terms
        for word in words:
            yield word, (key_term, _len)
        
    def reducer(self,key,value):
        
        d = collections.defaultdict(list)
        for v in value:
            d[key].append(v)
        yield key,d[key] 

#END STUDENT CODE531_INV_INDEX
        
if __name__ == '__main__':
    MRinvertedIndex.run() 

Overwriting invertedIndex.py


In [311]:
%%writefile similarity.py
#!~/anaconda2/bin/python
# -*- coding: utf-8 -*-

from __future__ import division
import collections
import re
import json
import math
# import numpy as np
import itertools
import mrjob
from mrjob.protocol import RawProtocol
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRsimilarity(MRJob):
  
  #START SUDENT CODE531_SIMILARITY
    
    MRJob.SORT_VALUES = True 
    def steps(self):

        JOBCONF_STEP1 = {}
        JOBCONF_STEP2 = { 
          ######### IMPORTANT: THIS WILL HAVE NO EFFECT IN -r local MODE. MUST USE -r hadoop FOR SORTING #############
            'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapreduce.partition.keycomparator.options':'-k1,1nr',
            
        }
        
        return [MRStep(jobconf=JOBCONF_STEP1,
                    mapper=self.mapper_pair_sim,
                    reducer=self.reducer_pair_sim)
                ,
                MRStep(jobconf=JOBCONF_STEP2,
                    mapper=None,   
                    reducer=self.reducer_sort)
                ]
            
    def mapper_pair_sim(self,_,line):
        line = line.strip()
        term,coterm = line.split("\t")
        coterm = json.loads(coterm)
        
        X = map(lambda x: x[0]+"."+str(x[1]) , coterm)
        
        # taking advantage of symmetry, output only (a,b), but not (b,a)
        # 'set' will output only the unique occurrences
        for subset in itertools.combinations(sorted(set(X)), 2):
            yield subset[0]+"."+subset[1], 1


    def reducer_pair_sim(self,key,value):
        Doc1, Doc1_len, Doc2, Doc2_len = key.split(".")
        doc1_len = int(Doc1_len)
        doc2_len = int(Doc2_len)
        t = sum(value)
        
        
        # calculate the similarity values
        jaccard = t / ( doc1_len + doc2_len - t )
        cosine = t * ((1/math.sqrt(doc1_len)) * (1/math.sqrt(doc2_len)))
        dice = (2*t) / (doc1_len + doc2_len)
        overlap = t / min(doc1_len, doc2_len)
        
        # Average the 4 similarities 
        avg = sum([jaccard,cosine,dice,overlap]) / 4
        
        yield [avg,jaccard,cosine,overlap,dice], (Doc1+" - "+Doc2)
    
    
    def reducer_sort(self,key,value):
        for v in value:
            yield key,v
            
#END SUDENT CODE531_SIMILARITY
  
if __name__ == '__main__':
    MRsimilarity.run()

Overwriting similarity.py


## HW5.3.1   Run Systems tests locally on small datasets (PHASE1) <a name="5.3.1"></a>  
[Back to Table of Contents](#TOC)

Complete 5.3 and systems test using the below test datasets. Phase 2 will focus on the entire Ngram dataset.

To help you through these tasks please verify that your code gives the results below (for stripes, inverted index, and pairwise similarities).

Test datasets:

* googlebooks-eng-all-5gram-20090715-0-filtered.txt [see below]
* atlas-boon-test [see below]
* stripe-docs-test [see below]


A large subset of the Google n-grams dataset

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket/folder on Dropbox and on s3:

https://www.dropbox.com/sh/tmqpc4o0xswhkvz/AACUifrl6wrMrlK6a3X3lZ9Ea?dl=0 

s3://filtered-5grams/

In particular, this bucket contains (~200) files (10Meg each) in the format:

	(ngram) \t (count) \t (pages_count) \t (books_count)

The next cell shows the first 10 lines of the googlebooks-eng-all-5gram-20090715-0-filtered.txt file.


__DISCLAIMER__: Each record is already a 5-gram. In real life, we would calculate the stripes cooccurrence data from the raw text by windowing over the raw text and not from the 5-gram preprocessed data (as we are doing here).  Calculatating pairs on this 5-gram is a little corrupt as we will be double counting cooccurences. Having said that this exercise can still pull out some simialr terms. 

#### 1: unit/systems first-10-lines

In [294]:
%%writefile googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt
A BILL FOR ESTABLISHING RELIGIOUS	59	59	54
A Biography of General George	92	90	74
A Case Study in Government	102	102	78
A Case Study of Female	447	447	327
A Case Study of Limited	55	55	43
A Child's Christmas in Wales	1099	1061	866
A Circumstantial Narrative of the	62	62	50
A City by the Sea	62	60	49
A Collection of Fairy Tales	123	117	80
A Collection of Forms of	116	103	82

Overwriting googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt


#### 2: unit/systems atlas-boon

In [295]:
%%writefile atlas-boon-systems-test.txt
atlas boon	50	50	50
boon cava dipped	10	10	10
atlas dipped	15	15	15

Overwriting atlas-boon-systems-test.txt


#### 3: unit/systems stripe-docs-test

Three terms, A,B,C and their corresponding stripe-docs of co-occurring terms

- DocA {X:20, Y:30, Z:5}
- DocB {X:100, Y:20}
- DocC {M:5, N:20, Z:5}

### (1) build stripes for all the test data sets - run the commands and insure that your output matches the output below

In [296]:
###########################################################################
# Make Stripes from ngrams for systems test 1
###########################################################################

!hdfs dfs rm --recursive systems_test_stripes_1
!python buildStripes.py -r local googlebooks-eng-all-5gram-20090715-0-filtered-first-10-lines.txt > systems_test_stripes_1

rm: Unknown command
Did you mean -rm?  This command begins with a dash.
No configs found; falling back on auto-configuration
Creating temp directory /tmp/buildStripes.root.20170213.072821.757803
Running step 1 of 1...
Streaming final output from /tmp/buildStripes.root.20170213.072821.757803/output...
Removing temp directory /tmp/buildStripes.root.20170213.072821.757803...


In [297]:
!cat systems_test_stripes_1

"a"	{"limited":55,"sea":62,"general":92,"female":447,"in":1201,"religious":59,"george":92,"biography":92,"city":62,"for":59,"tales":123,"government":102,"the":124,"forms":116,"wales":1099,"christmas":1099,"child's":1099,"collection":239,"by":62,"case":604,"circumstantial":62,"of":1011,"study":604,"bill":59,"establishing":59,"narrative":62,"fairy":123}
"bill"	{"a":59,"religious":59,"for":59,"establishing":59}
"biography"	{"a":92,"of":92,"george":92,"general":92}
"by"	{"a":62,"city":62,"the":62,"sea":62}
"case"	{"a":604,"limited":55,"government":102,"of":502,"study":604,"female":447,"in":102}
"child's"	{"a":1099,"wales":1099,"christmas":1099,"in":1099}
"christmas"	{"a":1099,"wales":1099,"in":1099,"child's":1099}
"circumstantial"	{"a":62,"of":62,"the":62,"narrative":62}
"city"	{"a":62,"the":62,"by":62,"sea":62}
"collection"	{"a":239,"forms":116,"fairy":123,"tales":123,"of":355}
"establishing"	{"a":59,"bill":59,"religious":59,"for":59}
"fairy"	{"a":123,"of":123,"tales":123,"collection":123

<pre>
"a"	{"limited": 55, "sea": 62, "general": 92, "female": 447, "in": 1201, "religious": 59, "george": 92, "biography": 92, "city": 62, "for": 59, "tales": 123, "child's": 1099, "forms": 116, "wales": 1099, "christmas": 1099, "government": 102, "collection": 239, "by": 62, "case": 604, "circumstantial": 62, "fairy": 123, "of": 1011, "study": 604, "bill": 59, "establishing": 59, "narrative": 62, "the": 124}
"bill"	{"a": 59, "religious": 59, "for": 59, "establishing": 59}
"biography"	{"a": 92, "of": 92, "george": 92, "general": 92}
"by"	{"a": 62, "city": 62, "the": 62, "sea": 62}
"case"	{"a": 604, "limited": 55, "government": 102, "of": 502, "study": 604, "female": 447, "in": 102}
"child's"	{"a": 1099, "wales": 1099, "christmas": 1099, "in": 1099}
"christmas"	{"a": 1099, "wales": 1099, "in": 1099, "child's": 1099}
"circumstantial"	{"a": 62, "of": 62, "the": 62, "narrative": 62}
"city"	{"a": 62, "the": 62, "by": 62, "sea": 62}
"collection"	{"a": 239, "of": 355, "fairy": 123, "tales": 123, "forms": 116}
"establishing"	{"a": 59, "bill": 59, "religious": 59, "for": 59}
"fairy"	{"a": 123, "of": 123, "tales": 123, "collection": 123}
"female"	{"a": 447, "case": 447, "study": 447, "of": 447}
"for"	{"a": 59, "bill": 59, "religious": 59, "establishing": 59}
"forms"	{"a": 116, "of": 232, "collection": 116}
"general"	{"a": 92, "of": 92, "george": 92, "biography": 92}
"george"	{"a": 92, "of": 92, "biography": 92, "general": 92}
"government"	{"a": 102, "case": 102, "study": 102, "in": 102}
"in"	{"a": 1201, "case": 102, "government": 102, "study": 102, "child's": 1099, "wales": 1099, "christmas": 1099}
"limited"	{"a": 55, "case": 55, "study": 55, "of": 55}
"narrative"	{"a": 62, "of": 62, "the": 62, "circumstantial": 62}
"of"	{"a": 1127, "case": 502, "circumstantial": 62, "george": 92, "limited": 55, "tales": 123, "collection": 471, "general": 92, "forms": 348, "female": 447, "narrative": 62, "study": 502, "fairy": 123, "the": 62, "biography": 92}
"religious"	{"a": 59, "bill": 59, "for": 59, "establishing": 59}
"sea"	{"a": 62, "city": 62, "the": 62, "by": 62}
"study"	{"a": 604, "case": 604, "limited": 55, "government": 102, "of": 502, "female": 447, "in": 102}
"tales"	{"a": 123, "of": 123, "fairy": 123, "collection": 123}
"the"	{"a": 124, "city": 62, "circumstantial": 62, "of": 62, "sea": 62, "narrative": 62, "by": 62}
"wales"	{"a": 1099, "in": 1099, "christmas": 1099, "child's": 1099}

</pre>

In [298]:
###########################################################################
# Make Stripes from ngrams for systems test 2
###########################################################################

!hdfs dfs rm --recursive systems_test_stripes_2
!python buildStripes.py -r local atlas-boon-systems-test.txt > systems_test_stripes_2

rm: Unknown command
Did you mean -rm?  This command begins with a dash.
No configs found; falling back on auto-configuration
Creating temp directory /tmp/buildStripes.root.20170213.072836.216996
Running step 1 of 1...
Streaming final output from /tmp/buildStripes.root.20170213.072836.216996/output...
Removing temp directory /tmp/buildStripes.root.20170213.072836.216996...


In [299]:
!cat systems_test_stripes_2

"atlas"	{"dipped":15,"boon":50}
"boon"	{"atlas":50,"dipped":10,"cava":10}
"cava"	{"dipped":10,"boon":10}
"dipped"	{"atlas":15,"boon":10,"cava":10}


<pre>
"atlas"   {"dipped": 15, "boon": 50}   
"boon"    {"atlas": 50, "dipped": 10, "cava": 10}   
"cava"    {"dipped": 10, "boon": 10} 
"dipped"  {"atlas": 15, "boon": 10, "cava": 10}
</pre>

In [300]:
########################################################################
# Stripes for systems test 3 (given, no need to build stripes)
########################################################################

with open("systems_test_stripes_3", "w") as f:
    f.writelines([
        '"DocA"\t{"X":20, "Y":30, "Z":5}\n',
        '"DocB"\t{"X":100, "Y":20}\n',  
        '"DocC"\t{"M":5, "N":20, "Z":5, "Y":1}\n'
    ])
!cat systems_test_stripes_3   

"DocA"	{"X":20, "Y":30, "Z":5}
"DocB"	{"X":100, "Y":20}
"DocC"	{"M":5, "N":20, "Z":5, "Y":1}


### (2) Build Inverted Index - run the commands and insure that your output matches the output below

In [301]:
!python invertedIndex.py -r local systems_test_stripes_1 > systems_test_index_1

No configs found; falling back on auto-configuration
Creating temp directory /tmp/invertedIndex.root.20170213.072857.238594
Running step 1 of 1...
Streaming final output from /tmp/invertedIndex.root.20170213.072857.238594/output...
Removing temp directory /tmp/invertedIndex.root.20170213.072857.238594...


In [302]:
!python invertedIndex.py -r local systems_test_stripes_2 > systems_test_index_2

No configs found; falling back on auto-configuration
Creating temp directory /tmp/invertedIndex.root.20170213.072900.428320
Running step 1 of 1...
Streaming final output from /tmp/invertedIndex.root.20170213.072900.428320/output...
Removing temp directory /tmp/invertedIndex.root.20170213.072900.428320...


In [303]:
!python invertedIndex.py -r local systems_test_stripes_3 > systems_test_index_3

No configs found; falling back on auto-configuration
Creating temp directory /tmp/invertedIndex.root.20170213.072903.458618
Running step 1 of 1...
Streaming final output from /tmp/invertedIndex.root.20170213.072903.458618/output...
Removing temp directory /tmp/invertedIndex.root.20170213.072903.458618...


In [306]:
##########################################################
# Pretty print systems tests for generating Inverted Index
##########################################################

import json

for i in range(1,4):
    print "—"*100
    print "Systems test ",i," - Inverted Index"
    print "—"*100  
    with open("systems_test_index_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            word,stripe = line.split("\t")
            stripe = json.loads(stripe)
            stripe.extend([["",""] for _ in xrange(3 - len(stripe))])
            print "{0:>16} |{1:>16} |{2:>16} |{3:>16}".format(
                (word), stripe[0][0]+" "+str(stripe[0][1]), stripe[1][0]+" "+str(stripe[1][1]), stripe[2][0]+" "+str(stripe[2][1]))
        


————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
             "a" |        "bill" 4 |   "biography" 4 |          "by" 4
          "bill" |          "a" 27 |"establishing" 4 |         "for" 4
     "biography" |          "a" 27 |     "general" 4 |      "george" 4
            "by" |          "a" 27 |        "city" 4 |         "sea" 4
          "case" |          "a" 27 |      "female" 4 |  "government" 4
       "child's" |          "a" 27 |   "christmas" 4 |          "in" 7
     "christmas" |          "a" 27 |     "child's" 4 |          "in" 7
"circumstantial" |          "a" 27 |   "narrative" 4 |         "of" 15
          "city" |          "a" 27 |          "by" 4 |         "sea" 4
    "collection" |          "a" 27 |       "fairy" 4 |       "forms" 3
  "establishing" |          "a" 27 |        "bill" 4 |

### Inverted Index

In [ ]:
————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
             "a" |          bill 4 |     biography 4 |            by 4
          "bill" |            a 27 |  establishing 4 |           for 4
     "biography" |            a 27 |       general 4 |        george 4
            "by" |            a 27 |          city 4 |           sea 4
          "case" |            a 27 |        female 4 |    government 4
       "child's" |            a 27 |     christmas 4 |            in 7
     "christmas" |            a 27 |       child's 4 |            in 7
"circumstantial" |            a 27 |     narrative 4 |           of 15
          "city" |            a 27 |            by 4 |           sea 4
    "collection" |            a 27 |         fairy 4 |         forms 3
  "establishing" |            a 27 |          bill 4 |           for 4
         "fairy" |            a 27 |    collection 5 |           of 15
        "female" |            a 27 |          case 7 |           of 15
           "for" |            a 27 |          bill 4 |  establishing 4
         "forms" |            a 27 |    collection 5 |           of 15
       "general" |            a 27 |     biography 4 |        george 4
        "george" |            a 27 |     biography 4 |       general 4
    "government" |            a 27 |          case 7 |            in 7
            "in" |            a 27 |          case 7 |       child's 4
       "limited" |            a 27 |          case 7 |           of 15
     "narrative" |            a 27 |circumstantial 4 |           of 15
            "of" |            a 27 |     biography 4 |          case 7
     "religious" |            a 27 |          bill 4 |  establishing 4
           "sea" |            a 27 |            by 4 |          city 4
         "study" |            a 27 |          case 7 |        female 4
         "tales" |            a 27 |    collection 5 |         fairy 4
           "the" |            a 27 |            by 4 |circumstantial 4
         "wales" |            a 27 |       child's 4 |     christmas 4
————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  2  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
         "atlas" |          boon 3 |        dipped 3 |                
          "boon" |         atlas 2 |          cava 2 |        dipped 3
          "cava" |          boon 3 |        dipped 3 |                
        "dipped" |         atlas 2 |          boon 3 |          cava 2
————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  3  - Inverted Index
————————————————————————————————————————————————————————————————————————————————————————————————————
             "M" |          DocC 4 |                 |                
             "N" |          DocC 4 |                 |                
             "X" |          DocA 3 |          DocB 2 |                
             "Y" |          DocA 3 |          DocB 2 |          DocC 4
             "Z" |          DocA 3 |          DocC 4 |                


### (3) Calculate similarities - run the commands and insure that your output matches the output below

#### NOTE: you must run in hadoop mode to generate sorted similarities

In [307]:
!python similarity.py -r hadoop systems_test_index_1 > systems_test_similarities_1

No configs found; falling back on auto-configuration
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/similarity.root.20170213.073031.946921
Copying local files to hdfs:///user/root/tmp/mrjob/similarity.root.20170213.073031.946921/files/...
Detected hadoop configuration property names that do not match hadoop version 2.6.0:
The have been translated as follows
 mapred.output.key.comparator.class: mapreduce.job.output.key.comparator.class
mapred.text.key.comparator.options: mapreduce.partition.keycomparator.options
mapred.text.key.partitioner.options: mapreduce.partition.keypartitioner.options
Running step 1 of 2...
  mapred.text.key.partitioner.options is deprecated. Instead, use mapreduce.partition.keypartiti

In [308]:
!python similarity.py -r hadoop systems_test_index_2 > systems_test_similarities_2

No configs found; falling back on auto-configuration
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/similarity.root.20170213.073151.795311
Copying local files to hdfs:///user/root/tmp/mrjob/similarity.root.20170213.073151.795311/files/...
Detected hadoop configuration property names that do not match hadoop version 2.6.0:
The have been translated as follows
 mapred.output.key.comparator.class: mapreduce.job.output.key.comparator.class
mapred.text.key.comparator.options: mapreduce.partition.keycomparator.options
mapred.text.key.partitioner.options: mapreduce.partition.keypartitioner.options
Running step 1 of 2...
  mapred.text.key.partitioner.options is deprecated. Instead, use mapreduce.partition.keypartiti

In [309]:
!python similarity.py -r hadoop systems_test_index_3 > systems_test_similarities_3

No configs found; falling back on auto-configuration
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/similarity.root.20170213.073316.826528
Copying local files to hdfs:///user/root/tmp/mrjob/similarity.root.20170213.073316.826528/files/...
Detected hadoop configuration property names that do not match hadoop version 2.6.0:
The have been translated as follows
 mapred.output.key.comparator.class: mapreduce.job.output.key.comparator.class
mapred.text.key.comparator.options: mapreduce.partition.keycomparator.options
mapred.text.key.partitioner.options: mapreduce.partition.keypartitioner.options
Running step 1 of 2...
  mapred.text.key.partitioner.options is deprecated. Instead, use mapreduce.partition.keypartiti

In [313]:
############################################
# Pretty print systems tests
############################################

import json
import ast
for i in range(1,4):
  print '—'*110
  print "Systems test ",i," - Similarity measures"
  print '—'*110
  print "{0:>15} |{1:>15} |{2:>15} |{3:>15} |{4:>15} |{5:>15}".format(
          "average", "pair", "cosine", "jaccard", "overlap", "dice")
  print '-'*110

  with open("systems_test_similarities_"+str(i),"r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            sims,stripe = line.split("\t")
            sims = ast.literal_eval(sims)
            stripe = json.loads(stripe)
            print "{0:>15f} |{1:>15} |{2:>15f} |{3:>15f} |{4:>15f} |{5:>15f}".format(
                sims[0], stripe, sims[2], sims[1], sims[3], sims[4])
 

——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
        average |           pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
       0.180200 |"the" - "wales" |       0.188982 |       0.100000 |       0.250000 |       0.181818
       0.223214 |"tales" - "wales" |       0.250000 |       0.142857 |       0.250000 |       0.250000
       0.365956 |"tales" - "the" |       0.377964 |       0.222222 |       0.500000 |       0.363636
       0.365956 |"study" - "wales" |       0.377964 |       0.222222 |       0.500000 |       0.363636
       0.255952 |"study" - "the" |       0.285714 |       0.166667 |       0.285714 |       0.285714
       0.365956 |"

### Pairwise Similairity 

In [ ]:
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  1  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
   average |                pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
  1.000000 |    female - limited |       1.000000 |       1.000000 |       1.000000 |       1.000000
  0.868292 |       fairy - forms |       0.866025 |       0.750000 |       1.000000 |       0.857143
  0.868292 |       forms - tales |       0.866025 |       0.750000 |       1.000000 |       0.857143
  0.830357 |        case - study |       0.857143 |       0.750000 |       0.857143 |       0.857143
  0.712500 | bill - establishing |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 |   christmas - wales |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 |circumstantial - narrative |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 |            by - sea |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 |           by - city |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 |     child's - wales |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 |  biography - george |       0.750000 |       0.600000 |       0.750000 |       0.750000
  0.712500 | child's - christmas |       0.750000 |       0.600000 |       0.750000 |       0.750000
  ...
  
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  2  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
   average |                pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
  1.000000 |        atlas - cava |       1.000000 |       1.000000 |       1.000000 |       1.000000
  0.625000 |       boon - dipped |       0.666667 |       0.500000 |       0.666667 |       0.666667
  0.389562 |       cava - dipped |       0.408248 |       0.250000 |       0.500000 |       0.400000
  0.389562 |         boon - cava |       0.408248 |       0.250000 |       0.500000 |       0.400000
  0.389562 |      atlas - dipped |       0.408248 |       0.250000 |       0.500000 |       0.400000
  0.389562 |        atlas - boon |       0.408248 |       0.250000 |       0.500000 |       0.400000
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
Systems test  3  - Similarity measures
——————————————————————————————————————————————————————————————————————————————————————————————————————————————
   average |                pair |         cosine |        jaccard |        overlap |           dice
--------------------------------------------------------------------------------------------------------------
  0.820791 |         DocA - DocB |       0.816497 |       0.666667 |       1.000000 |       0.800000
  0.553861 |         DocA - DocC |       0.577350 |       0.400000 |       0.666667 |       0.571429
  0.346722 |         DocB - DocC |       0.353553 |       0.200000 |       0.500000 |       0.333333


# === END OF PHASE 1 ===